In [1]:
%matplotlib inline

In [2]:
import logging
from datetime import datetime, timedelta

In [3]:
from nearest_geophys_point_finder import NearestGeophysPointFinder
logging.getLogger('nearest_geophys_point_finder').setLevel(logging.DEBUG)

In [4]:
# Setup proxy as required
GA_STAFF_WIFI = False

if GA_STAFF_WIFI:
    import os
    os.environ['http_proxy'] = 'http://proxy.inno.lan:3128'
    os.environ['https_proxy'] = 'http://proxy.inno.lan:3128'

In [5]:
def JW_metadata_filter(metadata_dict):
    '''
    Example function to filter datasets based on metadata values in metadata_dict
    This version applies John Wilford's filter conditions
    Returns True for match, False otherwise
    '''
    try:
        # Reject any datasets earlier than 1981
        if datetime.strptime(metadata_dict['acquisition_start_date'], '%d/%m/%Y') < datetime.strptime('01/01/1981', '%d/%m/%Y'):
            return False
            
        # Only accept GA/BMR/AGSO datasets between 1981 and 1992
        if (datetime.strptime(metadata_dict['acquisition_start_date'], '%d/%m/%Y') < datetime.strptime('01/01/1992', '%d/%m/%Y')
            and metadata_dict['client'] not in ['Geoscience Australia',
                                                'BMR',
                                                'AGSO',
                                                'GA'
                                                ]
            ):
                return False
    except ValueError:
        logger.warning('WARNING: Unhandled date format: {}'.format(metadata_dict['acquisition_start_date']))
        return False 
    
    return True        

In [6]:
# Instantiate NearestGeophysPointFinder
ngpf = NearestGeophysPointFinder()

Importing metadata CSV file C:\Users\u76345\git\geophys_utils_python3\examples\geophysics_line_nc_metadata.csv


In [7]:
# Set search parameters
coordinate_list=[(149, -35), (150, -36)] # List of coordinates to search
max_distance=0.2 # Maximum search radius in dataset native units (i.e. degrees)
points_required=10 # Points per dataset
keywords = ['geophysics', 'TMI'] # Keywords to match
variable_names = ['mag_awags'] # Variables to read
flight_lines_only=True

In [8]:
# Find all nearest points using above parameters
point_result_dict = ngpf.get_nearest_point_data(coordinate_list=coordinate_list,
                                         points_required=points_required, 
                                         max_distance=max_distance,
                                         keywords=keywords,
                                         metadata_filter_function=JW_metadata_filter,
                                         variable_names=variable_names,
                                         flight_lines_only=flight_lines_only
                                         )

Opening http://dapds00.nci.org.au/thredds/dodsC/uc0/rr2_dev/rcb547/AWAGS_Levelled_Line_Databases/awags_survey_reformat/netcdf/P1152MAG.nc
Excluding points in tie-lines
10 points near (149, -35) found in http://dapds00.nci.org.au/thredds/dodsC/uc0/rr2_dev/rcb547/AWAGS_Levelled_Line_Databases/awags_survey_reformat/netcdf/P1152MAG.nc
10 points near (150, -36) found in http://dapds00.nci.org.au/thredds/dodsC/uc0/rr2_dev/rcb547/AWAGS_Levelled_Line_Databases/awags_survey_reformat/netcdf/P1152MAG.nc
Opening http://dapds00.nci.org.au/thredds/dodsC/uc0/rr2_dev/rcb547/AWAGS_Levelled_Line_Databases/mag_database_reformat_adjusted/netcdf/GSNSW_P1218MAG.nc
Excluding points in tie-lines
10 points near (149, -35) found in http://dapds00.nci.org.au/thredds/dodsC/uc0/rr2_dev/rcb547/AWAGS_Levelled_Line_Databases/mag_database_reformat_adjusted/netcdf/GSNSW_P1218MAG.nc
10 points near (150, -36) found in http://dapds00.nci.org.au/thredds/dodsC/uc0/rr2_dev/rcb547/AWAGS_Levelled_Line_Databases/mag_database_re

In [9]:
# Count points and datasets
for coordinate in sorted(point_result_dict.keys()):
    point_list = point_result_dict[coordinate]
    dataset_set = set()
    for point_dict in point_list:
        if set(variable_names) < set(point_dict.keys()): # Only count results with all required variables
            dataset_set.add(point_dict['file_path'])
    print('A total of {} valid points were found near {} in {} datasets'.format(len(point_list), coordinate, len(dataset_set)))

A total of 50 valid points were found near (149, -35) in 5 datasets
A total of 30 valid points were found near (150, -36) in 3 datasets


In [10]:
# Print results in tab-delimited format
field_list = ['search_coordinate', 'coordinate', 'distance'] + variable_names + ['survey_id', 'client', 'title']

print('\t'.join(field_list))
for coordinate in sorted(point_result_dict.keys()):
    point_list = point_result_dict[coordinate]
    for point_dict in point_list[0:points_required]:
        point_dict['search_coordinate'] = coordinate
        if set(variable_names) < set(point_dict.keys()): # Only show results with all required variables
            print('\t'.join([str(point_dict.get(field)) 
                             for field in field_list]))

search_coordinate	coordinate	distance	mag_awags	survey_id	client	title
(149, -35)	(148.932424, -34.977558)	5.234500931190665e-05	45.779987	650	AGSO	Goulburn, NSW, 1996
(149, -35)	(148.93251, -34.977554)	5.8008620044722445e-05	45.910854	650	AGSO	Goulburn, NSW, 1996
(149, -35)	(148.932338, -34.977562)	0.00012713772061180662	45.746773	650	AGSO	Goulburn, NSW, 1996
(149, -35)	(148.932595, -34.97755)	0.00013439121996190135	46.133034	650	AGSO	Goulburn, NSW, 1996
(149, -35)	(148.932252, -34.977566)	0.00020934421415168972	45.827816	650	AGSO	Goulburn, NSW, 1996
(149, -35)	(148.932681, -34.977547)	0.00021656638704725954	46.45287	650	AGSO	Goulburn, NSW, 1996
(149, -35)	(148.9999858, -35.0002253)	0.00022574704870455172	-13.487463	1253	GSNSW	Murrumbateman, NSW, 2010 Detailed Airborne Survey
(149, -35)	(149.0000453, -35.0002258)	0.00023029921841410015	-13.352302	1253	GSNSW	Murrumbateman, NSW, 2010 Detailed Airborne Survey
(149, -35)	(148.9999263, -35.0002247)	0.0002364778636549526	-13.634345	1253	GSN